In [1]:
import os 
import pandas as pd
import re
import pandas as pd
import csv

In [100]:
file_lists = []

for curdir,dirs,files in os.walk('./数据集'):
    for file in files:
        if file.endswith('.txt'):
            file = os.path.join(curdir,file)
            file_lists.append(file)

# for file in file_lists:
#     print(file)
print(len(file_lists))

778


In [101]:
patts = {'生产总值/地区生产总值/GDP/地区生产总值（GDP）/地区生产总值（GDP）达到/生产总值实现':(
    r'生产总值.*?(\d+.\d+)(\w+元).*?(增长|下降).*?(\d+.\d+\w*%)',(1,2)),
        '按可比价格计算，同比增长/比上年增长/按可比价计算，增长/按可比价计算， GDP总量增长/，与上年比增速/同比增长':(
            r'生产总值.*?(\d+.\d+)(\w+元).*?((增长|下降).*?(\d+.\d*.*?%*))',(3,)),
        '第一产业增加值/第一产业实现/一产业实现增加值':(
            r'第一产业增加值.*?(\d+.\d+)(\w+元).*?(增长|下降).*?(\d+.\d+\w*(%|元))',(1,2)),
        '第二产业增加值/第二产业实现/二产业实现增加值':(
            r'第二产业增加值.*?(\d+.\d+)(\w+元).*?(增长|下降).*?(\d+.\d+\w*(%|元))',(1,2)),
        '第三产业增加值/第三产业实现/三产业实现增加值':(
            r'第三产业增加值.*?(\d+.\d+)(\w+元).*?(增长|下降).*?(\d+.\d+\w*(%|元))',(1,2)),
        '人均GDP':(r'人均.*?(GDP|生产).*?(\d+).*?(\D*元).*?',(2,3)),
        '规模以上工业增加值/规模以上工业（XXX）增加值/工业增加值':(
            r'工业和建筑业.*?\n*?.*?(规模以上|工业)(增加值).*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(3,4)),
        '规模以上工业增加值增长率':(
            r'工业和建筑业.*?\n*?.*?(规模以上|工业)(增加值).*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(5,)),
        '全社会固定资产投资/固定资产投资/固定资产投资(省口径)':(
            r'固定资产投资.*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(1,2)),
        '社会固定资产投资增长率':(r'固定资产投资.*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(3,)),
        '房地产投资/房地产开发投资/房地产开发完成投资/房地产开发投资完成':(
            r'(房地产|房地产开发).*?投资.*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(2,3)),
        '房地产开发投资同比增长':(
            r'(房地产|房地产开发).*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(4,)),
        '社会消费品零售总额/社会消费品零售额':(r'(消费品).*?总额.*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(2,3)),
        '社会消费品零售总额增长率':(r'(消费品).*?总额.*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(4,)),
        '地方公共财政预算收入/一般预算收入/一般公共预算收入/地方公共预算收入':(
            r'(财政|一般).*?收入.*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(2,3)),
        '地方公共预算收入同比增长':(
            r'(财政|一般).*?收入.*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(4,)),
        '财政总支出/财政支出/一般预算支出/一般公共预算支出':(
            r'(财政支出|预算支出).*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(2,3)),
        '政府性基金支出/基金支出':(r'基金支出.*?收入.*?(\d+.\d+)(.*?元)',(1,2)),
        '金融机构年末各项存款余额/人民币各项存款余额':(
            r'(存款余额).*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(2,3)),
        '金融机构年末各项贷款余额/人民币各项贷款余额':(
            r'(贷款余额).*?(\d+.\d+)(.*?元).*?((增长|下降).*?(\d+.\d*.*?%*))',(2,3)),
        '户籍总人口/户籍人口':(r'(总人口|户籍人口).*?(\d+.\d+)(.*?人)',(2,3)),
        '常住人口/常住总人口':(r'(常住人口).*?(\d+.\d+)(.*?人)',(2,3)),
        '城镇住户人均可支配收入/城镇常住居民人均可支配收入/城镇常住居民人均总收入/城镇居民人均可支配收入':(
            r'(城镇).*?可支配收入.*?(\d+.\d+)(.*?元)',(2,3)),
        '农村居民人均可支配收入/农民人均可支配收入':(
            r'(农村).*?可支配收入.*?(\d+.\d+)(.*?元)',(2,3))
        }

In [102]:
citys_infos = {}
citys_errors = {}
print(len(file_lists))
for file in file_lists:
    with open(file, 'r') as f:
        s = f.read()
        infos = {}
        errors = {}
        for key,val in patts.items():
            infos[key] = []
            if len(val[1]) == 2:
                try:
                    num = re.search(val[0],s).group(val[1][0])
                    char = re.search(val[0],s).group(val[1][1])
                    infos[key].append([num,char])
                except Exception as e:
                    errors[key] = e
            elif len(val[1]) == 1:
                try:
                    char = re.search(val[0],s).group(val[1][0])
                    infos[key].append([char,None])
                except Exception as e:
                    errors[key] = e                
        citys_infos[file] = infos
        citys_errors[file] = errors
print(len(citys_infos))
print(len(citys_errors))

778
778
778


In [115]:
with open('数据8.txt','w+',encoding='utf-8') as f:
    for city in citys_infos:
        # 获取分类文件夹的名称作为分类标签
        label = city[city.rfind(
                os.path.sep) + 1:].rsplit('.txt')[0]
        for key,val in citys_infos[city].items():
#             print('****************')
#             print(val)
#             print('***********')
            data = label+'::'+key
            for vl in val:
                for v in vl:
                    if v:
#                         print(v)
                        data +='::'+v
            f.write(data+'\n')

In [116]:
datas = pd.read_csv('数据8.txt', sep='::',encoding='utf_8_sig',
                    engine='python',header=None)
datas.head()

,0,1,2,3
0,安徽省-亳州市-利辛县-2015,生产总值GDP,173.8,亿元
1,安徽省-亳州市-利辛县-2015,GDP总量增长,增长8.7%,None
2,安徽省-亳州市-利辛县-2015,第一产业增加值,44.3,亿元
3,安徽省-亳州市-利辛县-2015,第二产业增加值,48.0,亿元
4,安徽省-亳州市-利辛县-2015,第三产业增加值,81.5,亿元


In [105]:
datas.to_csv('政府整合信息06.csv', mode='a', 
                encoding='utf_8_sig', index=False, sep=',')